# Convert and Optimize TimeSformer with OpenVINO™
TimeSformer(from Time-Space Transformer) devised and proposed in [Is Space-Time Attention All You Need for Video Understanding?](https://arxiv.org/abs/2102.05095) uses 